<a href="https://colab.research.google.com/github/deegrayce/Election_analysis/blob/main/Geopy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from geopy.geocoders import GoogleV3

# Loading dataset
df = pd.read_csv("/content/OGUN_crosschecked.csv")
df.head()

,State,LGA,Ward,PU-Code,PU-Name,Accredited_Voters,Registered_Voters,Results_Found,Transcription_Count,Result_Sheet_Stamped,Result_Sheet_Corrected,Result_Sheet_Invalid,Result_Sheet_Unclear,Result_Sheet_Unsigned,APC,LP,PDP,NNPP,Results_File
0,OGUN,ABEOKUTA NORTH,IKEREKU,27-01-01-001,O.U.S PRY. SCHOOL IKEREKU I,64,607,True,-1,False,False,False,False,UNKNOWN,0,0,0,0,https://docs.inecelectionresults.net/elections...
1,OGUN,ABEOKUTA NORTH,IKEREKU,27-01-01-002,O.U.S PRY. SCHOOL IKEREKU II,137,775,True,-1,True,False,False,False,UNKNOWN,40,3,7,0,https://docs.inecelectionresults.net/elections...
2,OGUN,ABEOKUTA NORTH,IKEREKU,27-01-01-003,NEAR HOLY TRINITY CHURCH I,114,776,True,-1,False,True,False,False,UNKNOWN,66,1,28,0,https://docs.inecelectionresults.net/elections...
3,OGUN,ABEOKUTA NORTH,IKEREKU,27-01-01-004,NEAR HOLY TRINITY CHURCH II,94,635,True,-1,False,True,False,False,UNKNOWN,58,2,13,0,https://docs.inecelectionresults.net/elections...
4,OGUN,ABEOKUTA NORTH,IKEREKU,27-01-01-005,NEAR ITA OGBONI IDOMAPA I,149,750,True,-1,False,True,False,False,UNKNOWN,80,1,54,0,https://docs.inecelectionresults.net/elections...


In [ ]:
import requests
import pandas as pd
from geopy.distance import geodesic

# Define function to get coordinates
def get_coordinates(address, api_key):
    base_url = "https://api.opencagedata.com/geocode/v1/json"
    params = {
        "q": address,
        "key": api_key
    }
    response = requests.get(base_url, params=params)
    if response.status_code == 200 and response.json()['results']:
        location = response.json()['results'][0]['geometry']
        return location['lat'], location['lng']
    else:
        return None, None

# Load dataset
df = pd.read_csv('/content/OGUN_crosschecked.csv')

# Initialize API key
api_key = '7a75ce5331d74e0faffe895947e5ce6c'

# Add columns for latitude and longitude
df['latitude'] = None
df['longitude'] = None

# Get coordinates for each address
for index, row in df.iterrows():
    address = row['Ward']  # Replace 'address' with the correct column name
    lat, lng = get_coordinates(address, api_key)
    df.at[index, 'latitude'] = lat
    df.at[index, 'longitude'] = lng

# Fill missing values with an average or a fixed location
df['latitude'].fillna(df['latitude'].mean(), inplace=True)
df['longitude'].fillna(df['longitude'].mean(), inplace=True)

# Save the updated dataset
df.to_csv('updated_dataset.csv', index=False)


In [ ]:
# Define function to calculate distance
def calculate_distance(coord1, coord2):
    return geodesic(coord1, coord2).kilometers


def find_neighbours(df, radius=1):
    neighbours = {}
    for i, unit in df.iterrows():
        unit_coord = (unit['latitude'], unit['longitude'])
        neighbours[unit['PU-Name']] = []
        for j, other_unit in df.iterrows():
            if i != j:
                other_coord = (other_unit['latitude'], other_unit['longitude'])
                distance = calculate_distance(unit_coord, other_coord)
                if distance <= radius:
                    neighbours[unit['PU-Name']].append(other_unit['PU-Name'])
    return neighbours


In [ ]:
# Finding neighbours
neighbours = find_neighbours(df)

# Defining function to calculate outlier scores
def calculate_outlier_scores(df, neighbours):
    scores = []
    for i, unit in df.iterrows():
        unit_name = unit['PU-Name']
        for party in ['APC', 'LP', 'PDP', 'NNPP']:  # List of party columns
            score = 0
            neighbour_votes = [df[df['PU-Name'] == n][party].values[0] for n in neighbours[unit_name]]
            if neighbour_votes:
                score = abs(unit[party] - sum(neighbour_votes) / len(neighbour_votes))
            scores.append({
                'unit': unit_name,
                'party': party,
                'score': score
            })
    return scores

# Calculating outlier scores
outlier_scores = calculate_outlier_scores(df, neighbours)

# Sorting scores
sorted_scores = sorted(outlier_scores, key=lambda x: x['score'], reverse=True)

# Saving sorted scores to a new DataFrame and export to CSV
sorted_scores_df = pd.DataFrame(sorted_scores)
sorted_scores_df.to_csv('sorted_outlier_scores.csv', index=False)

# Printing top 3 outliers
print(sorted_scores_df.head(3))